<a href="https://colab.research.google.com/github/neethipoonacha/EIP3/blob/master/eip3_session2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# load ascii text and covert to lowercase
filename = "/content/gdrive/My Drive/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  163344
Total Vocab:  58


In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  163244


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [8]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0726 04:43:53.977230 140425579214720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 04:43:54.013309 140425579214720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 04:43:54.021646 140425579214720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 04:43:54.377609 140425579214720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0726 04:43:54.388933 

In [0]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [10]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

W0726 04:44:01.952933 140425579214720 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
163244/163244 [==============================] - 1058s 6ms/step - loss: 2.9877

Epoch 00001: loss improved from inf to 2.98770, saving model to weights-improvement-01-2.9877.hdf5
Epoch 2/20
163244/163244 [==============================] - 1059s 6ms/step - loss: 2.8107

Epoch 00002: loss improved from 2.98770 to 2.81075, saving model to weights-improvement-02-2.8107.hdf5
Epoch 3/20
163244/163244 [==============================] - 1058s 6ms/step - loss: 2.7341

Epoch 00003: loss improved from 2.81075 to 2.73414, saving model to weights-improvement-03-2.7341.hdf5
Epoch 4/20
163244/163244 [==============================] - 1054s 6ms/step - loss: 2.6750

Epoch 00004: loss improved from 2.73414 to 2.67495, saving model to weights-improvement-04-2.6750.hdf5
Epoch 5/20
163244/163244 [==============================] - 1049s 6ms/step - loss: 2.6127

Epoch 00005: loss improved from 2.67495 to 2.61266, saving model to weights-improvement-05-2.6127.hdf5
Epoch 6/20
163244/163244 [========

In [0]:
f = drive.CreateFile()
f.SetContentFile("weights-improvement-19-2.0902.hdf5")
f.Upload()

In [0]:
# load the network weights
filename = "weights-improvement-19-2.0902.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [18]:
# pick a random seed
import sys
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" result seemed to follow,
except a little shaking among the distant green leaves.

as there seemed to "
 the whrt woen i soone  she would bete to be a garter or the tooed an it was to the whrt of the woodd 

ani the hert tas toin to the whrt wae the tas oo the war of the woide. 
'leee th the the garter an i cad'' said the march hare.

'ie tou doe't kn whth the soiet ' said the march hare.

'ie tou doe't kn whth the soiet ' said the march hare.

'ie tou doe't kn whth the soiet ' said the march hare.

'ie tou doe't kn whth the soiet ' said the march hare.

'ie tou doe't kn whth the soiet ' said the march hare.

'ie tou doe't kn whth the soiet ' said the march hare.

'ie tou doe't kn whth the soiet ' said the march hare.

'ie tou doe't kn whth the soiet ' said the march hare.

'ie tou doe't kn whth the soiet ' said the march hare.

'ie tou doe't kn whth the soiet ' said the march hare.

'ie tou doe't kn whth the soiet ' said the march hare.

'ie tou doe't kn whth the soiet ' said t

In [23]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "/content/gdrive/My Drive/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
#add padding to the text
from keras.preprocessing.sequence import pad_sequences
# pad sequence

#padded = pad_sequences(raw_text, maxlen=5)
#print(padded)
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
#maxlen = max(dataY)

#print ("Max length of character is ",dataX,":",max(dataX))
print ("Max length of character Y ",dataY,":",max(dataY))
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
padded = pad_sequences(y, maxlen=max(dataY))
print("padded",padded)
print("y is ",y)
print("X is ",X)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
#model.add(Dropout(0.2))
model.add(Dense(padded.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="/content/gdrive/My Drive/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

#f = drive.CreateFile()
#f.SetContentFile(filepath)
#f.Upload()
# fit the model
model.fit(X, padded, epochs=100, batch_size=64, callbacks=callbacks_list)



Total Characters:  510
Total Vocab:  31
Max length of character Y  [1, 12, 22, 7, 1, 22, 21, 11, 13, 1, 22, 24, 1, 26, 29, 17, 11, 13, 1, 25, 16, 13, 1, 16, 9, 12, 1, 23, 13, 13, 23, 13, 12, 1, 17, 21, 26, 22, 1, 26, 16, 13, 0, 10, 22, 22, 18, 1, 16, 13, 24, 1, 25, 17, 25, 26, 13, 24, 1, 29, 9, 25, 1, 24, 13, 9, 12, 17, 21, 15, 5, 1, 10, 27, 26, 1, 17, 26, 1, 16, 9, 12, 1, 21, 22, 1, 23, 17, 11, 26, 27, 24, 13, 25, 1, 22, 24, 1, 11, 22, 21, 28, 13, 24, 25, 9, 26, 17, 22, 21, 25, 1, 17, 21, 0, 17, 26, 5, 1, 2, 9, 21, 12, 1, 29, 16, 9, 26, 1, 17, 25, 1, 26, 16, 13, 1, 27, 25, 13, 1, 22, 14, 1, 9, 1, 10, 22, 22, 18, 5, 2, 1, 26, 16, 22, 27, 15, 16, 26, 1, 9, 19, 17, 11, 13, 1, 2, 29, 17, 26, 16, 22, 27, 26, 1, 23, 17, 11, 26, 27, 24, 13, 25, 1, 22, 24, 0, 11, 22, 21, 28, 13, 24, 25, 9, 26, 17, 22, 21, 25, 8, 2, 0, 25, 22, 1, 25, 16, 13, 1, 29, 9, 25, 1, 11, 22, 21, 25, 17, 12, 13, 24, 17, 21, 15, 1, 17, 21, 1, 16, 13, 24, 1, 22, 29, 21, 1, 20, 17, 21, 12, 1, 3, 9, 25, 1, 29, 13, 19, 19, 1

In [27]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "/content/gdrive/My Drive/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
padded = pad_sequences(y, maxlen=max(dataY))
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
#model.add(Dropout(0.2))
model.add(Dense(padded.shape[1], activation='softmax'))
# load the network weights
filename = "/content/gdrive/My Drive/weights-improvement-100-0.1640-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
#padded = pad_sequences(pattern, maxlen=max(dataY))
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Total Characters:  510
Total Vocab:  31
Total Patterns:  410
Seed:
" he
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use  "
ne
?
anni) 
sgntfgs
?khbd
 vhsgnts
ohbstpdr
np
bnmudpr?shnmr: rm
hdm
hmmf
nmm
v??
rm
vdcc
rg
nndb??r
sn
occp
sn

?m
sssk

?r
rrw,bg?h
vnt
c
annpsggsgddssd
nt
cnp

np
sdddd
rgn
ohbs
rp
sn
cm

sst) 
sn
vggs
?hhbd
 vhsgnts
ohbstpdr
np
bnmudpr?shnmr: hm
rdd ??mc
vgbshhhrmr: rm
gd?
nm

nm
svhb
)
vgdsgdp
sgd
okd?rtpd
ne
l?ihmf
?
c?hrw,bg?hm
vntkc
ad
vnpsg
sgd
spntakd
ne
fdsshmf
to
sgdttkd
ne
ldishmf
to
sgdttkd
ne
ldsshmf
to
sgdttkd
ne
fdsshmf
to
sgdttkd
ne
ldishmf
to
sgdttkd
ne
ldsshm
?
c?hrw,bg?hm
vntkc
ad
vnpsg
sgd
spntakd
ne
fdsshmf
to
sgdttkd
ne
ldishmf
to
sgdttkd
ne
ldsshmf
to
sgdttkd
ne
fdsshmf
to
sgdttkd
ne
ldishmf
to
sgdttkd
ne
ldsshm
?
c?hrw,bg?hm
vntkc
ad
vnpsg
sgd
spntakd
ne
fdsshmf
to
sgdttkd
ne
ldishmf
to
sgdttkd
ne
ldsshmf
to
sgdttkd
ne
fdsshmf
to
sgdttkd
ne
ldishmf
to
sgdttkd
ne
ldsshm
?
c?hrw,bg?hm
vntkc
ad